# Part 1

In [1]:
# Taken from day 17 and modified for today.
def render_map(map_):
    ''' Render a section of the [infinite] map. '''
    min_x = min(coord[0] for coord in map_.keys())
    max_x = max(coord[0] for coord in map_.keys())
    min_y = min(coord[1] for coord in map_.keys())
    max_y = max(coord[1] for coord in map_.keys())
    render = ''
    for y in range(min_y, max_y+1):
        render += ''.join('X' if x==0 and y==0 else str(map_.get((x,y),' '))
                          for x in range(min_x, max_x+1))
        render += '\n'
    return render

def print_map(map_):
    print(render_map(map_))

In [2]:
def create_map():
    ''' Create a blank map with the intial room at 0,0. '''
    return {
        (-1,-1): '#', ( 0,-1): '?', ( 1,-1): '#',
        (-1, 0): '?', ( 0, 0): '.', ( 1, 0): '?',
        (-1, 1): '#', ( 0, 1): '?', ( 1, 1): '#',
    }

In [3]:
def fill_walls(map_):
    ''' Replace new map where ? is replaced with #. '''
    return {(x, y):('#' if char == '?' else char) for (x,y), char in map_.items()}

In [4]:
test_map = create_map()
print_map(test_map)
print_map(fill_walls(test_map))

#?#
?X?
#?#

###
#X#
###



In [5]:
class Symbol:
    def __init__(self, name):
        self.name = name
        
    def __repr__(self):
        return 'Symbol({})'.format(self.name)

OPEN = Symbol('OPEN')
PIPE = Symbol('PIPE')
CLOSE = Symbol('CLOSE')
    
def lex_regex(regex):
    ''' Split regex into a sequence of tokens. Ignores start and end specifiers: ^$. '''
    tokens = list()
    current = list()
    for char in regex:
        if char in 'NESW':
            current.append(char)
            continue
        if current:
            tokens.append(''.join(current))
            current = list()
        if char == '(':
            tokens.append(OPEN)
        elif char == '|':
            tokens.append(PIPE)
        elif char == ')':
            tokens.append(CLOSE)
    return tokens

In [6]:
def debug_lex(regex):
    print(lex_regex(regex))
    
debug_lex('^WNE$')

['WNE']


In [7]:
debug_lex('^N(E|W)N$')

['N', Symbol(OPEN), 'E', Symbol(PIPE), 'W', Symbol(CLOSE), 'N']


In [8]:
debug_lex('^NN(EE|WW(SS|NN))EE$')

['NN', Symbol(OPEN), 'EE', Symbol(PIPE), 'WW', Symbol(OPEN), 'SS', Symbol(PIPE), 'NN', Symbol(CLOSE), Symbol(CLOSE), 'EE']


In [9]:
class RegexList(list):
    ''' A list that contains regex strings and groups. '''
    def append(self, x):
        assert isinstance(x, (str, RegexGroup))
        super().append(x)
    
    def __str__(self, indent=''):
        render = '{}RegexList[\n'.format(indent)
        for x in self:
            if isinstance(x, str):
                render += '  {}{}\n'.format(indent, x)
            else:
                render += x.__str__(indent + '  ')
        render += '{}]\n'.format(indent)
        return render
        
class RegexGroup:
    ''' A container for RegexList items. '''
    def __init__(self):
        self._items = list()

    def __str__(self, indent=''):
        render = '{}RegexGroup{{\n'.format(indent)
        for i in self._items:
            render += i.__str__(indent + '  ')
        render += '{}}}\n'.format(indent)
        return render
        
    def add_item(self, item):
        assert isinstance(item, RegexList)
        self._items.append(item)
    
    @property
    def items(self):
        return self._items

In [10]:
rg = RegexGroup()
rl1 = RegexList()
rl1.append('NN')
rg.add_item(rl1)
rl2 = RegexList()
rl2.append('SS')
rg.add_item(rl2)
print(rg)

RegexGroup{
  RegexList[
    NN
  ]
  RegexList[
    SS
  ]
}



In [11]:
rl = RegexList()
rl.append('EE')
rl.append(rg)
rl.append('WW')
print(rl)

RegexList[
  EE
  RegexGroup{
    RegexList[
      NN
    ]
    RegexList[
      SS
    ]
  }
  WW
]



In [12]:
def parse_tokens(tokens):
    ''' Parse tokens and return a RegexList. '''
    # Create a stack of regex lists and a stack of groups
    root = RegexList()
    regex_lists = [root]
    groups = []
    for token in tokens:
        if isinstance(token, Symbol):
            if token is OPEN:
                # Create a new group and add it to current regex list
                group = RegexGroup()
                groups.append(group)
                regex_lists[-1].append(group)
                # Create a new regex list. Add it to group and put it on 
                # the stack
                rl = RegexList()
                regex_lists.append(rl)
                group.add_item(rl)
            elif token is PIPE:
                # We have finished a regex list. Pop it off the stack and create
                # a new regex list.
                regex_list = regex_lists.pop()
                rl = RegexList()
                groups[-1].add_item(rl)
                regex_lists.append(rl)
            elif token is CLOSE:
                # We have finished a regex list. Pop it off the stack.
                regex_list = regex_lists.pop()
                # We have finished a group. Pop it off the stack.
                group = groups.pop()                
        else:
            # Token is a plain string. Append to current list.
            regex_lists[-1].append(token)
    return root

def debug_parse(regex):
    tokens = lex_regex(regex)
    print(tokens)
    regex_list = parse_tokens(tokens)
    print(regex_list)

In [13]:
debug_parse('^NN(EE|WW)SS$')

['NN', Symbol(OPEN), 'EE', Symbol(PIPE), 'WW', Symbol(CLOSE), 'SS']
RegexList[
  NN
  RegexGroup{
    RegexList[
      EE
    ]
    RegexList[
      WW
    ]
  }
  SS
]



In [14]:
debug_parse('^NN(EE|WW(NN|SS)WW)SS$')

['NN', Symbol(OPEN), 'EE', Symbol(PIPE), 'WW', Symbol(OPEN), 'NN', Symbol(PIPE), 'SS', Symbol(CLOSE), 'WW', Symbol(CLOSE), 'SS']
RegexList[
  NN
  RegexGroup{
    RegexList[
      EE
    ]
    RegexList[
      WW
      RegexGroup{
        RegexList[
          NN
        ]
        RegexList[
          SS
        ]
      }
      WW
    ]
  }
  SS
]



In [15]:
def plot_regex_string(regex, map_, points):
    ''' Given a regex string that contains only directions (no groups), 
    plot the corresponding walls, doors, rooms, and question marks on 
    the map from the given starting points. Returns the endpoints. '''

    def question(qx, qy):
        ''' If qx,qy not in map, then set it to question mark. '''
        if (qx,qy) not in map_:
            map_[qx,qy] = '?'

    endpoints = list()
    for (x,y) in points:
        for char in regex:
            if char == 'N':
                map_[x,  y-1] = '-'
                map_[x  ,y-2] = '.'
                map_[x-1,y-3] = '#'
                map_[x+1,y-3] = '#'
                question(x-1, y-2)
                question(x+1, y-2)
                question(x  , y-3)
                y -= 2
            elif char == 'S':
                map_[x,  y+1] = '-'
                map_[x  ,y+2] = '.'
                map_[x-1,y+3] = '#'
                map_[x+1,y+3] = '#'
                question(x-1, y+2)
                question(x+1, y+2)
                question(x  , y+3)
                y += 2
            elif char == 'W':
                map_[x-1,y  ] = '|'
                map_[x-2,y  ] = '.'
                map_[x-3,y-1] = '#'
                map_[x-3,y+1] = '#'
                question(x-2, y-1)
                question(x-2, y+1)
                question(x-3, y  )
                x -= 2
            elif char == 'E':
                map_[x+1,y  ] = '|'
                map_[x+2,y  ] = '.'
                map_[x+3,y-1] = '#'
                map_[x+3,y+1] = '#'
                question(x+2, y-1)
                question(x+2, y+1)
                question(x+3, y  )
                x += 2
            else:
                raise Exception('Unexpected char="{}"'.format(char))
        endpoints.append((x, y))
    return list(set(endpoints))

def debug_plot_regex_string(regex, points):
    map_ = create_map()
    points = plot_regex_string(regex, map_, points)
    print_map(map_)
    return points    

In [16]:
# Plot N and S. (There are some missing walls because the second
# plot isn't attached to the first one, but that's OK.)
debug_plot_regex_string('NSS', points=[(0,0),(4,2)])

#?#    
?.?    
#-# #?#
?X? ?.?
#-#  - 
?.? ?.?
#?# #-#
    ?.?
    #?#



[(4, 4), (0, 2)]

In [17]:
def plot_regex_group(item, map_, points):
    ''' Plot a regex group starting from each of the given points. 
    Return the new endpoints. '''
    endpoints = list()
    for regex_list in item.items:
        endpoints += plot_regex_list(regex_list, map_, points)
    return list(set(endpoints))

def plot_regex_list(regex_list, map_, points):
    ''' Plot each item of a regex list, starting at the given endpoints. Returns
    a list of new endpoints. '''
    for item in regex_list:
        if isinstance(item, str):
            points = plot_regex_string(item, map_, points)
        elif isinstance(item, RegexGroup):
            points = plot_regex_group(item, map_, points)
        else:
            raise Exception('Invalid item in RegexList')
    return list(set(points))

def debug_plot(regex, fill=False):
    tokens = lex_regex(regex)
    regex_list = parse_tokens(tokens)
    map_ = create_map()
    endpoints = plot_regex_list(regex_list, map_, points=[(0,0)])
    if fill:
        print_map(fill_walls(map_))
    else:
        print_map(map_)
    return endpoints

In [18]:
# Plot a cross.
debug_plot('^EEWWWWEENNSSSS$')

    #?#    
    ?.?    
    #-#    
    ?.?    
#?#?#-#?#?#
?.|.|X|.|.?
#?#?#-#?#?#
    ?.?    
    #-#    
    ?.?    
    #?#    



[(0, 4)]

In [19]:
# Plot a regex that ends with a group. This should make a T shap.
debug_plot('^NN(EE|WW)$')

#?#?#?#?#?#
?.|.|.|.|.?
#?#?#-#?#?#
    ?.?    
    #-#    
    ?X?    
    #?#    



[(4, -4), (-4, -4)]

In [20]:
# A plot with directions after a group. This should make an M shape.
debug_plot('^NN(EE|WW)SS$')

#?#?#?#?#?#
?.|.|.|.|.?
#-#?#-#?#-#
?.? ?.? ?.?
#-# #-# #-#
?.? ?X? ?.?
#?# #?# #?#



[(4, 0), (-4, 0)]

In [21]:
# A plot with nested groups.
debug_plot('^NN(EE|WW(NN|SS))SS$')

#?#        
?.?        
#-#        
?.?        
#-#?#?#?#?#
?.|.|.|.|.?
#-#?#-#?#-#
?.? ?.? ?.?
#-# #-# #-#
?.? ?X? ?.?
#-# #?# #?#
?.?        
#-#        
?.?        
#?#        



[(-4, 4), (-4, -4), (4, 0)]

In [22]:
# First example from problem.
debug_plot('^WNE$')

#?#?#
?.|.?
#-#?#
?.|X?
#?#?#



[(0, -2)]

In [23]:
# Second example from problem.
debug_plot('^ENWWW(NEEE|SSE(EE|N))$', fill=True)

#########
#.|.|.|.#
#-#######
#.|.|.|.#
#-#####-#
#.#.#X|.#
#-#-#####
#.|.|.|.#
#########



[(2, -4), (-2, 0), (2, 2)]

In [24]:
# The next example problem. This one has empty group items!
debug_plot('^ENNWSWW(NEWS|)SSSEEN(WNSE|)EE(SWEN|)NNN$', fill=True)

###########
#.|.#.|.#.#
#-###-#-#-#
#.|.|.#.#.#
#-#####-#-#
#.#.#X|.#.#
#-#-#####-#
#.#.|.|.|.#
#-###-###-#
#.|.|.#.|.#
###########



[(4, -4)]

In [25]:
def plot(regex):
    ''' Plot a regex and return a map. '''
    tokens = lex_regex(regex)
    regex_list = parse_tokens(tokens)
    map_ = create_map()
    plot_regex_list(regex_list, map_, points=[(0,0)])
    map_ = fill_walls(map_)
    return map_

In [35]:
def map_size(map_, debug=False):
    ''' Compute the distance to the room farthest from the origin. '''
    stack = [(0,0)]
    def fill_map(x, y):
        ''' Fill in the distance to the squares adjacent to x,y. '''
        dist = map_[x,y] + 1
        if map_[x-1,y] == '|':
            val = map_[x-2,y]
            if val == '.' or val > dist:
                map_[x-2,y] = dist
                stack.append((x-2,y))
        if map_[x+1,y] == '|':
            val = map_[x+2,y]
            if val == '.' or val > dist:
                map_[x+2,y] = dist
                stack.append((x+2,y))
        if map_[x,y-1] == '-':
            val = map_[x,y-2]
            if val == '.' or val > dist:
                map_[x,y-2] = dist
                stack.append((x,y-2))
        if map_[x,y+1] == '-':
            val = map_[x,y+2]
            if val == '.' or val > dist:
                map_[x,y+2] = dist
                stack.append((x,y+2))
    map_ = dict(map_)
    map_[0,0] = 0
    for x, y in stack:
        fill_map(x,y)
    if debug:
        print_map(map_)
    values = [val for val in map_.values() if isinstance(val, int)]
    return max(values)

def debug_map_size(regex):
    map_ = plot(regex)
    print_map(map_)
    return map_size(map_, debug=True)

In [36]:
debug_map_size('^WNE$')

#####
#.|.#
#-###
#.|X#
#####

#####
#2|3#
#-###
#1|X#
#####



3

In [37]:
map_size(plot('^ENWWW(NEEE|SSE(EE|N))$'))

10

In [38]:
map_size(plot('^ENNWSWW(NEWS|)SSSEEN(WNSE|)EE(SWEN|)NNN$'))

18

In [39]:
map_size(plot('^ESSWWN(E|NNENN(EESS(WNSE|)SSS|WWWSSSSE(SW|NNNE)))$'))

23

In [40]:
map_size(plot('^WSSEESWWWNW(S|NENNEEEENN(ESSSSW(NWSW|SSEN)|WSWWN(E|WWS(E|SS))))$'))

31

In [41]:
%%time
# This was really slow–would not finish after several minutes–the first time
# I ran it, then I added deduplication for end points in the plot*() methods 
# and now it runs almost instantly!
with open('input.txt') as input_:
    regex = input_.read().strip()
map_ = plot(regex)

CPU times: user 87.1 ms, sys: 3.66 ms, total: 90.8 ms
Wall time: 89.5 ms


In [45]:
# For debugging
with open('output.txt', 'w') as output_:
    output_.write(render_map(map_))

In [46]:
map_size(map_)

3835

# Part 2

In [56]:
def count_paths(map_, threshold):
    ''' Compute how many rooms are farther than threshold units away from
    the origin. Very similar to map_size() except instead of returning max
    distance it filters by distance and counts the number of rooms that
    remain. '''
    stack = [(0,0)]
    def fill_map(x, y):
        ''' Fill in the distance to the squares adjacent to x,y. '''
        dist = map_[x,y] + 1
        if map_[x-1,y] == '|':
            val = map_[x-2,y]
            if val == '.' or val > dist:
                map_[x-2,y] = dist
                stack.append((x-2,y))
        if map_[x+1,y] == '|':
            val = map_[x+2,y]
            if val == '.' or val > dist:
                map_[x+2,y] = dist
                stack.append((x+2,y))
        if map_[x,y-1] == '-':
            val = map_[x,y-2]
            if val == '.' or val > dist:
                map_[x,y-2] = dist
                stack.append((x,y-2))
        if map_[x,y+1] == '-':
            val = map_[x,y+2]
            if val == '.' or val > dist:
                map_[x,y+2] = dist
                stack.append((x,y+2))
    map_ = dict(map_)
    map_[0,0] = 0
    for x, y in stack:
        fill_map(x,y)
    values = [val for val in map_.values() if isinstance(val, int) and val >= 1000]
    return len(values)

In [57]:
with open('input.txt') as input_:
    regex = input_.read().strip()
map_ = plot(regex)

In [58]:
count_paths(map_, threshold=1000)

8520